# Setup

In [1]:
%%capture
! pip install datasets metrics transformers wandb

In [2]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from transformers import BertTokenizerFast, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import BertModel, BertTokenizer
from transformers.file_utils import ModelOutput
from datasets import Dataset
from torch import nn
import pandas as pd
import torch
import gc

# Custom BERT model

In [3]:
class BERTShallowFC(nn.Module):
    def __init__(self,
                 model_name='bert-base-uncased',
                 device='cuda',
                 num_classes=3):
        super(BERTShallowFC, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.fc = nn.Linear(768, num_classes).to(device)  # (BERT CLS size, num_classes)
        self.softmax = nn.LogSoftmax(dim=1).to(device)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            output_hidden_states=True)
        x = self.fc(outputs[0][:, 0, :])
        c = self.softmax(x)

        # Clean cache
        gc.collect()
        torch.cuda.empty_cache()
        del outputs

        # Compute loss
        loss = None
        if labels is not None:
            ce_loss = nn.CrossEntropyLoss()
            loss = ce_loss(c, labels)

        return ModelOutput({
            'loss': loss,
            'last_hidden_state': c
        })

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)
model = BERTShallowFC(model_name)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Pre-procesing

In [6]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels

    inputs = tokenizer(
        batch["1"],
        max_length=50,
        padding="max_length",
        truncation=True,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    labels = list(map(lambda x: int(x.split('__label__')[1]), batch['0']))
    batch["label"] = labels

    return batch

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate f1 using sklearn's function
  f_score = f1_score(labels, preds, pos_label=0, average='binary')
  accuracy = accuracy_score(labels, preds)
  precision = precision_score(labels, preds, pos_label=0, average='binary')
  recall = recall_score(labels, preds, pos_label=0, average='binary')
  return {
      'f1_score': f_score,
      'accuracy': accuracy,
      'precision': precision,
      'recall': recall
  }
    
tokenize_batch_size = 524282
df = pd.read_csv('/content/drive/MyDrive/PoliTo/Deep Natural Language Procesing/Project/labeledEligibilitySample1000000.csv', sep='\t', header=None)

In [7]:
train_ds = Dataset.from_pandas(df)

train_ds = train_ds.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=tokenize_batch_size,
    remove_columns=["0", "1"]
)

train_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [8]:
train_testvalid = train_ds.train_test_split(train_size=0.9)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

# Training and evaluation

In [9]:
training_args = TrainingArguments(
    output_dir='./ShallowFC_results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=256,  # batch size per device during training
    per_device_eval_batch_size=256,   # batch size for evaluation
    fp16=True,                       # enable fp16 apex training
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=400,               # log & save weights each logging_steps
    save_steps=400,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
    report_to="wandb"
)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_testvalid['train'],         # training dataset
    eval_dataset=test_valid['train'],          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

trainer.train()

preds = trainer.predict(test_valid['test'])

preds[2]

Using amp half precision backend
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 900000
  Num Epochs = 2
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 7032
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,F1 Score,Accuracy,Precision,Recall
400,0.434400,0.276827,0.895025,0.893180,0.880812,0.909705
800,0.260500,0.223237,0.911446,0.913020,0.929366,0.894203
1200,0.221000,0.208205,0.921502,0.922720,0.937384,0.906149
1600,0.203800,0.200439,0.924955,0.923020,0.903275,0.947701
2000,0.189400,0.180294,0.932618,0.932160,0.927423,0.937872
2400,0.182000,0.176770,0.933472,0.934960,0.956483,0.911543
2800,0.176800,0.165528,0.938832,0.939220,0.945972,0.931799
3200,0.164400,0.156884,0.941926,0.941880,0.942265,0.941588
3600,0.154900,0.154317,0.943133,0.943860,0.956641,0.930001
4000,0.138000,0.148969,0.946777,0.946600,0.944743,0.948819


***** Running Evaluation *****
  Num examples = 50000
  Batch size = 256
Saving model checkpoint to ./ShallowFC_results/checkpoint-400
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 256
Saving model checkpoint to ./ShallowFC_results/checkpoint-800
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 256
Saving model checkpoint to ./ShallowFC_results/checkpoint-1200
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 256
Saving model checkpoint to ./ShallowFC_results/checkpoint-1600
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 256
Saving model checkpoint to ./ShallowFC_results/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, 

{'test_accuracy': 0.95352,
 'test_f1_score': 0.9532845541529309,
 'test_loss': 0.12702326476573944,
 'test_precision': 0.9614402140858777,
 'test_recall': 0.9452660952760614,
 'test_runtime': 91.4832,
 'test_samples_per_second': 546.549,
 'test_steps_per_second': 2.142}

In [10]:
import torch
torch.save(trainer.model, '/content/drive/MyDrive/PoliTo/Deep Natural Language Procesing/Project/model.model')